In [4]:
import pickle

def save_pickle(object, path):
    # Open a file in binary write mode
    with open(path, 'wb') as file:
        # Serialize the object and write it to the file
        pickle.dump(object, file)

def load_pickle(path):
    # Open the file in binary read mode
    with open(path, 'rb') as file:
    # Deserialize the object from the file
        loaded_data = pickle.load(file)

    return loaded_data

In [3]:
#!/usr/bin/env python

# Import necessary libraries
import cv2
import numpy as np
from dipy.align.imwarp import SymmetricDiffeomorphicRegistration
from dipy.align.metrics import CCMetric
from dipy.align.transforms import AffineTransform2D
from dipy.align.imaffine import AffineRegistration

def compute_diffeomorphic_mapping_dipy(y: np.ndarray, x: np.ndarray, sigma_diff=20, radius=20):
    """
    Compute diffeomorphic mapping using DIPY.
    
    Parameters:
        y (ndarray): Reference image.
        x (ndarray): Moving image to be registered.
        sigma_diff (int, optional): Standard deviation for the CCMetric. Default is 20.
        radius (int, optional): Radius for the CCMetric. Default is 20.

    Returns:
        mapping: A mapping object containing the transformation information.
    """
    if y.shape != x.shape:
        raise ValueError("Reference image (y) and moving image (x) must have the same shape.")
    
    # Initialize the AffineRegistration and AffineTransform2D objects
    affreg = AffineRegistration()
    transform = AffineTransform2D()

    # Perform the optimization procedure with two images reference_image and moving_image
    # params0 is set to None as we don't have initial parameters
    affine = affreg.optimize(y, x, transform, params0=None)

    # Define the metric and registration object
    metric = CCMetric(2, sigma_diff=sigma_diff, radius=radius)
    sdr = SymmetricDiffeomorphicRegistration(metric)

    # Perform the diffeomorphic registration using a pre-alignment from affine registration
    mapping = sdr.optimize(y, x, prealign=affine.affine)

    return mapping  


/opt/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
import os
from concurrent.futures import ProcessPoolExecutor


def process_crop(fixed_file, moving_file, current_crops_dir_fixed, current_crops_dir_moving, checkpoint_dir):
    """
    Loads a pair of fixed and moving crops from their respective directories.

    Args:
        fixed_file (str): Filename of the fixed crop.
        moving_file (str): Filename of the moving crop.
        current_crops_dir_fixed (str): Directory where fixed crops are stored.
        current_crops_dir_moving (str): Directory where moving crops are stored.

    Returns:
        tuple: A tuple containing the fixed crop and the moving crop.
    """
    fixed_crop = load_pickle(os.path.join(current_crops_dir_fixed, fixed_file))
    moving_crop = load_pickle(os.path.join(current_crops_dir_moving, moving_file))

    checkpoint_path = os.path.join(checkpoint_dir, f'mapping_{fixed_crop[0][0]}_{fixed_crop[0][1]}.pkl')
    
    if os.path.exists(checkpoint_path):
        # Load mapping from checkpoint if it exists
        mapping_diffeomorphic = load_pickle(checkpoint_path)
        print(f"Loaded checkpoint for i={fixed_crop[0][0]}_{fixed_crop[0][1]}")
    else:
        fixed_crop_dapi = fixed_crop[1][:, :, 2]
        mov_crop_dapi = moving_crop[1][:, :, 2]

        if fixed_crop_dapi.shape != mov_crop_dapi.shape:
            return None
        else:
            mapping_diffeomorphic = compute_diffeomorphic_mapping_dipy(fixed_crop_dapi, mov_crop_dapi)
            # Save the computed mapping
            save_pickle(mapping_diffeomorphic, checkpoint_path)
            print(f"Saved checkpoint for i={fixed_crop[0][0]}_{fixed_crop[0][1]}")
            
    return fixed_crop, moving_crop

In [8]:
current_crops_dir_fixed = '/Volumes/scratch/DIMA/chiodin/tests/images_h2000_w2000/data/crops/2024.08.30_DAPI_PPARgamma_PDL1/785R1_DAPI_PPARgamma_PDL1'
current_crops_dir_moving = '/Volumes/scratch/DIMA/chiodin/tests/images_h2000_w2000/data/crops/2024.08.31_DAPI_PD1_SRSF3/785R1_DAPI_PD1_SRSF3'
checkpoint_dir = '/Volumes/scratch/DIMA/chiodin/tests/images_h2000_w2000/data/mappings/2024.08.31_DAPI_PD1_SRSF3/785R1_DAPI_PD1_SRSF3'
max_workers=2

In [9]:
fixed_files = [os.path.join(current_crops_dir_fixed, file) for file in os.listdir(current_crops_dir_fixed)]
moving_files = [os.path.join(current_crops_dir_moving, file) for file in os.listdir(current_crops_dir_moving)]

In [17]:
if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

mappings = []

# Use ProcessPoolExecutor for parallel processing
with ProcessPoolExecutor(max_workers=max_workers) as executor:
    # Submit tasks for each crop
    futures = [
        executor.submit(process_crop, fixed_crop, moving_crop, current_crops_dir_fixed, current_crops_dir_moving, checkpoint_dir)
        for fixed_crop, moving_crop in zip(fixed_files, moving_files)
    ]

    # # Collect the results as they complete
    # for future in futures:
    #     mappings.append(future.result()["mapping"])


Process SpawnProcess-8:
Process SpawnProcess-7:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/miniconda3/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/miniconda3/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/miniconda3/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/miniconda3/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/miniconda3/lib/python3.12/concurrent/futures/process.py", line 251, in _process_worker
    call_item = call_queue.get(block=True)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/lib/python3.12/concurrent/futures/process.py", line 251, in _process_worker
    call_item = call_queue.get(block=True)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/lib/python